Autor: Jakub Białek

W związku z faktem, że dr. Krzysztof Kontek "ma prawo jako badacz do szukania anomalii tylko w jedną stronę", ja poczułem się w obowiązku sprawdzić co będzie jeśli zastosujemy tę samą metodologię dla obu kandydatów. W związku z tym, że p. Kontek nie podzielił się kodem, napisałem go sam. Głównym pytaniem jakie stawia ta analiza jest więc: czy stosując te same definicje anomalii dla obu kandydatów, metodologia wskaże podobną liczbę komisji, w których doszło do nadużyć? Odpowiedź jest twierdząca.

Pewna różnica wyników jest efektem różnic w sposobie grupowanie komisji - próbowałem odtworzyć to co opisał p. Kontek, ale opisany algorytm jest niejednoznaczny. Grupy, które składają się z komisji o tym samem kodzie pocztowym będą identyczne. Różnice pojawią się w łączeniu komisji o różnych kodach. Różnice te nie powinny mieć wpływu na odpowiedź na główne pytanie tej analizy -w końcu obaj kandydaci byli analizowani w tych samych grupach, z których znaczna część pokrywała się z grupami Kontka.

Serdecznie zapraszam wszystkich kolegów z branży do weryfikacji i poprawek tej analizy - w końcu nikt nie jest nieomylny. Sprawdzałem poszczególne obliczenia "na piechotę" dla danego wycinka danych i wyniki się zgadzały, ale nadal potrzebna jest druga para oczu. 



In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

#### Wczytanie danych o obwodach, czyszczenie, grupowanie

In [2]:
obwody = pd.read_csv('./obwody_glosowania_utf8.csv', sep=';')

In [3]:
obwody = obwody[['Gmina','TERYT gminy','Numer', 'Kod pocztowy', 'Typ obszaru', 'Wyborcy']]

In [4]:
obwody['Typ obszaru'].unique()

array(['miasto', 'wieś', 'miasto i wieś', 'dzielnica w m.st. Warszawa',
       'statek', 'zagranica'], dtype=object)

In [5]:
# usuń statek i zagranicę
obwody = obwody[obwody['Typ obszaru'].isin(('miasto', 'wieś', 'miasto i wieś', 'dzielnica w m.st. Warszawa'))]

In [6]:
obwody = obwody.dropna()

In [7]:
obwody.head(2)

,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy
0,m. Bolesławiec,20101.0,1,59-700,miasto,1637.0
1,m. Bolesławiec,20101.0,2,59-700,miasto,1286.0


### Klastrowanie

Z artykułu:
1. Początkowe grupowanie oparto na pierwszych dwóch cyfrach kodu pocztowego (np. „30” dla obszaru Krakowa).
2. Jeżeli powstała grupa zawierała od 10 do 16 komisji, została zaakceptowana bez zmian.
3. Grupy liczące mniej niż 10 komisji odłożono do późniejszego łączenia.
4. Grupy przekraczające 16 komisji dzielono rekurencyjnie, dodając kolejne cyfry kodu
    pocztowego (np. z „30” → „301” → „3011” i dalej, aż do pełnych pięciu cyfr).
5. Pozostałe małe grupy łączono z najbliższymi sąsiadami mającymi ten sam prefiks, przy czym
    priorytetem była ciągłość przestrzenna i zrównoważona liczebność grup.
    W odróżnieniu od wcześniejszego podejścia, które dopuszczało grupy o wielkości 10–25 komisji,
    niniejsze badanie przyjęło węższy zakres docelowy: od 10 do 16 komisji na grupę
   
   
Komentarz: Algorytm z artykułu nie jest deterministyczny (niejednoznacznie wskazuje co się dzieje z grupami liczącymi mniej niż 10 komisji do ponownego połączenia) nie da się go więc odtworzyć na podstawie jego opisu. Dlatego właśnie rzetelne artykułu zawierają kod źródłowy umożliwiający reprodukcję. Autor nie podzielił się ani kodem, ani grupowaniem. Jeśli to zrobi, zaktualizuję grupowanie.

Na szczęście nie powinno mieć to większego znaczenia dla celu tej analizy - a jest nim sprawdzenie czy metodologia Kontka wskazuje również anomalie na korzyść Trzaskowskiego i jaka jest ich skala.

In [8]:
import pandas as pd
from collections import defaultdict

def assign_groups(df: pd.DataFrame,
                  code_col: str = "kod",
                  min_size: int = 10,
                  max_size: int = 16,
) -> pd.DataFrame:
    """
    Zwraca kopię df z nową kolumną `group_id` zawierającą numer grupy
    spełniającej warunki 10 ≤ n ≤ 16.
    """

    work = df.copy()
    work["_code_str"] = work[code_col].astype(str).str.zfill(5)

    accepted = {}                # {prefix: list[index]}
    small     = {}               # prefixy < min_size (do późniejszego łączenia)

    # 1. Rekurencyjne dzielenie dużych grup
    def split(prefix: str, idxs: list[int], depth: int):
        size = len(idxs)

        # a) gotowa grupa
        if min_size <= size <= max_size or depth == 5:
            accepted[prefix] = idxs
            return
        # b) za mała – zapisz do późniejszego łączenia
        if size < min_size:
            small[prefix] = idxs
            return
        # c) za duża – dziel głębiej
        next_depth = depth + 1
        sub_prefixes = work.loc[idxs, "_code_str"].str[:next_depth]
        for sub_pref, sub_idxs in work.loc[idxs].groupby(sub_prefixes).groups.items():
            split(sub_pref, list(sub_idxs), next_depth)

    # start od 2-cyfrowych
    for pref2, grp in work.groupby(work["_code_str"].str[:2]).groups.items():
        split(pref2, list(grp), depth=2)

    # 2. Łączenie małych grup w obrębie tych samych 2-cyfrowych prefiksów
    buckets = defaultdict(list)          # {pref2: [(pref, idxs), ...]}
    for p, idxs in small.items():
        buckets[p[:2]].append((p, idxs))

    for pref2, lst in buckets.items():
        # uporządkuj rosnąco wg wartości prefiksu → przybliżenie "sąsiedztwa"
        lst.sort(key=lambda x: int(x[0]))
        buf_idx, buf_pref = [], []

        for p, idxs in lst:
            buf_idx.extend(idxs)
            buf_pref.append(p)

            if len(buf_idx) >= min_size:
                # jeśli przypadkiem > max_size, tnij w kawałki
                while len(buf_idx) > max_size:
                    accepted[f"{pref2}_{len(accepted)}"] = buf_idx[:max_size]
                    buf_idx = buf_idx[max_size:]
                accepted["+".join(buf_pref)] = buf_idx
                buf_idx, buf_pref = [], []

        # resztka < 10 – dociągamy do ostatniej grupy z tego prefiksu
        if buf_idx:
            *_, last_key = [k for k in accepted.keys() if k.startswith(pref2)]
            accepted[last_key].extend(buf_idx)

    # 3. Ostateczne przypisanie numerów
    idx2gid = {}
    for gid, (_, lst) in enumerate(accepted.items()):
        for i in lst:
            idx2gid[i] = gid

    work["group_id"] = work.index.map(idx2gid)

    return work.drop(columns="_code_str")

In [9]:
obwody['kod'] = obwody['Kod pocztowy'].str.replace('-','').astype(int) 
obwody = assign_groups(obwody, code_col='kod')

Z artykułu:

"Większość grup spełniła założony docelowy rozmiar: 1 386 grup (62,8%) zawierało od 10 do 16
komisji, a 2 017 grup (91,3%) zawierało od 6 do 30 komisji. Większe grupy zazwyczaj odpowiadały 5
obszarom miejskim — na przykład takim jak Toruń czy Włocławek, gdzie pojedynczy kod pocztowy
obejmował całe miasto"

In [10]:
# między 10 a 16
sum((obwody['group_id'].value_counts()>=10) & (obwody['group_id'].value_counts()<=16))

1223

In [11]:
# między 6 a 30
sum((obwody['group_id'].value_counts()>=6) & (obwody['group_id'].value_counts()<=30))

1925

In [12]:
# duże grupy
obwody[obwody['group_id']==2150]

,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id
4100,m. Toruń,46301.0,1,87-100,miasto,1579.0,87100,2150
4101,m. Toruń,46301.0,2,87-100,miasto,1767.0,87100,2150
4102,m. Toruń,46301.0,3,87-100,miasto,649.0,87100,2150
4103,m. Toruń,46301.0,4,87-100,miasto,1839.0,87100,2150
4104,m. Toruń,46301.0,5,87-100,miasto,1404.0,87100,2150
...,...,...,...,...,...,...,...,...
4221,m. Toruń,46301.0,122,87-100,miasto,68.0,87100,2150
4222,m. Toruń,46301.0,123,87-100,miasto,46.0,87100,2150
4223,m. Toruń,46301.0,124,87-100,miasto,121.0,87100,2150
4224,m. Toruń,46301.0,125,87-100,miasto,31.0,87100,2150


In [13]:
# usun grupy mniej niz 3-komisjowe
obwody['liczba komisji w grupie']  = obwody.groupby('group_id').transform('count')['kod']
obwody = obwody[obwody['liczba komisji w grupie']>3]

In [14]:
len(obwody)

30553

# Wyniki głosowania

In [15]:
#https://wybory.gov.pl/prezydent2025/pl/dane_w_arkuszach
p = pd.read_csv('./protokoly_po_obwodach_utf8.csv', sep=';')
d = pd.read_csv('./protokoly_po_obwodach_w_drugiej_turze_utf8.csv', sep=';')


In [16]:
keep = ['Teryt Gminy', 'Nr komisji', 'Gmina', 'TRZASKOWSKI Rafał Kazimierz', 'NAWROCKI Karol Tadeusz']

In [17]:
p = p[keep].dropna()
d = d[keep].dropna()

In [18]:
p.head(2)

,Teryt Gminy,Nr komisji,Gmina,TRZASKOWSKI Rafał Kazimierz,NAWROCKI Karol Tadeusz
0,20101.0,1,m. Bolesławiec,361.0,287.0
1,20101.0,2,m. Bolesławiec,381.0,228.0


Łączymy wyniki z 1 i 2 tury, oznaczamy odpowiednio suffixem _1, _2.

In [19]:
df = pd.merge(
    left=p, 
    right=d,
    how='inner',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['Teryt Gminy', 'Nr komisji'],
    suffixes = ('_1', '_2')
)

In [20]:
df.head(2)

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2
0,20101.0,1,m. Bolesławiec,361.0,287.0,m. Bolesławiec,596.0,582.0
1,20101.0,2,m. Bolesławiec,381.0,228.0,m. Bolesławiec,517.0,386.0


In [21]:
# polacz z obwodami

In [22]:
df = pd.merge(
    left=df, 
    right=obwody,
    how='inner',
    left_on=['Teryt Gminy', 'Nr komisji'],
    right_on=['TERYT gminy', 'Numer'],
)


In [23]:
from copy import copy
df = copy(df.dropna())

# Reprodukcja Analizy z artykułu dla obu kandydatów

## 1. Nadmierne poparcie dla Karola Nawrockiego (względem mediany w ramach lokalnej grupy)

Za artykułem: w ramach każdej grupy obliczono mediany oraz odchylenia bezwzględne od mediany (MAD), dla każdej komisji obliczono wskaźnik odchylenia od mediany

$k=\frac{X-mediana}{MAD}$

gdzie:

X - wynik w drugiej turze kandydata

mediana - mediana wyników kandydata w drugiej turze

MAD - odchelenie bezwzględne mediany kandydata w drugiej turze

In [24]:
kandydat_A = "TRZASKOWSKI Rafał Kazimierz"
kandydat_B = "NAWROCKI Karol Tadeusz"

In [25]:
# mediana w grupie
df[kandydat_A + '_mediana_2'] = df.groupby('group_id')[kandydat_A+'_2'].transform('median')
df[kandydat_B + '_mediana_2'] = df.groupby('group_id')[kandydat_B+'_2'].transform('median')

In [26]:
# MAD w grupie
from scipy.stats import median_abs_deviation
def mad(series):
    """Median Absolute Deviation"""
    return median_abs_deviation(series)

df[kandydat_A + "_MAD_2"] = df.groupby("group_id")[kandydat_A+'_2'].transform(mad)
df[kandydat_B + "_MAD_2"] = df.groupby("group_id")[kandydat_B+'_2'].transform(mad)

In [27]:
# wspolczynnik k

In [28]:
df[kandydat_A +'_k_analiza_1'] = (df[kandydat_A +'_2'] - df[kandydat_A + '_mediana_2'])/df[kandydat_A + '_MAD_2']
df[kandydat_B +'_k_analiza_1'] = (df[kandydat_B +'_2'] - df[kandydat_B + '_mediana_2'])/df[kandydat_B + '_MAD_2']

In [29]:
k = 3

In [30]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 2794)

In [31]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 2015)

In [32]:
k = 2

In [33]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_1']>k)

('TRZASKOWSKI Rafał Kazimierz', 4551)

In [34]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_1']>k)

('NAWROCKI Karol Tadeusz', 3750)

## WYNIKI:
Dla k=2, takich komisji, w których "za duże" poparcie ma Nawrocki jest 3750, a Trzaskowski 4551.

Przykładowa grupa, w której są komisje gdzie obaj kandydaci mają "nadmierne" poparcie. Jak widać wysokie K wynika tylko i wyłącznie z rozmiaru komisji. Im większa komisja, tym większa szansa na anomalię wykrytą w ten sposób.

In [35]:
df[df['group_id']==1568]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1
45,20103.0,1,gm. Gromadka,306.0,312.0,gm. Gromadka,423.0,570.0,gm. Gromadka,20103.0,1,59-706,wieś,1628.0,59706,1568,6,91.5,177.0,11.5,32.0,28.826087,12.28125
46,20103.0,2,gm. Gromadka,47.0,208.0,gm. Gromadka,108.0,354.0,gm. Gromadka,20103.0,2,59-706,wieś,770.0,59706,1568,6,91.5,177.0,11.5,32.0,1.434783,5.53125
47,20103.0,3,gm. Gromadka,45.0,59.0,gm. Gromadka,73.0,132.0,gm. Gromadka,20103.0,3,59-706,wieś,305.0,59706,1568,6,91.5,177.0,11.5,32.0,-1.608696,-1.40625
48,20103.0,4,gm. Gromadka,60.0,91.0,gm. Gromadka,85.0,167.0,gm. Gromadka,20103.0,4,59-706,wieś,431.0,59706,1568,6,91.5,177.0,11.5,32.0,-0.565217,-0.31250
49,20103.0,5,gm. Gromadka,49.0,64.0,gm. Gromadka,87.0,158.0,gm. Gromadka,20103.0,5,59-706,wieś,420.0,59706,1568,6,91.5,177.0,11.5,32.0,-0.391304,-0.59375
50,20103.0,6,gm. Gromadka,52.0,111.0,gm. Gromadka,96.0,187.0,gm. Gromadka,20103.0,6,59-706,wieś,436.0,59706,1568,6,91.5,177.0,11.5,32.0,0.391304,0.31250


In [36]:
# zapisz anomalie na korzyść kandydatów do późniejszego sumowania
df[kandydat_A + "_anomalia_1"] = df[kandydat_A +'_k_analiza_1']>2
df[kandydat_B + "_anomalia_1"] = df[kandydat_B +'_k_analiza_1']>2

## 2. Nadmierny względny wzrost poparcia dla Karola Nawrockiego między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla Rafała Trzaskowskiego w tej samej grupie lokalnej;


Nie podano wprost jak to było obliczone więc kolejno:
1. Dla danego kandydata obliczam względny wzrost między pierwszą a drugą turą (dzieląc wynik z drugiej przez wynik z pierwszej)
2. Następnie odnoszę go do wzrostu drugiego kandydata - liczę różnicę między względnymi wzrostami.
3. Dalej tak jak w pierwszym typie anomalii - dla tych różnic liczę medianę grupy, MAD grupy oraz odchylenie k w komisji.

In [37]:
# wzgledny wzrost między pierwszą a drugą turą
df[kandydat_A+ '_wzrost'] = df[kandydat_A+'_2']/df[kandydat_A+'_1']
df[kandydat_B+ '_wzrost'] = df[kandydat_B+'_2']/df[kandydat_B+'_1']

In [38]:
# roznica wzglednego wzrostu miedzy kandydatami
df['roz_wzr_' + kandydat_A] =  df[kandydat_A + '_wzrost']  - df[kandydat_B + '_wzrost'] # wzrost A w porównaniu do B
df['roz_wzr_' + kandydat_B] =  df[kandydat_B + '_wzrost']  - df[kandydat_A + '_wzrost'] # wzrost B w porównaniu do A

In [39]:
# mediana i mad różnicy względnego wzrostu poparcia
df['roz_wzr_' + kandydat_A +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_A].transform('median')
df['roz_wzr_' + kandydat_A + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_A].transform(mad)

In [40]:
df['roz_wzr_' + kandydat_B +'_mediana'] = df.groupby('group_id')['roz_wzr_' + kandydat_B].transform('median')
df['roz_wzr_' + kandydat_B + '_MAD'] =    df.groupby('group_id')['roz_wzr_' + kandydat_B].transform(mad)

In [41]:
df[kandydat_A +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_A] - df['roz_wzr_' + kandydat_A +'_mediana'])/df['roz_wzr_' + kandydat_A + '_MAD']

In [42]:
df[kandydat_B +'_k_analiza_2'] = (df['roz_wzr_' + kandydat_B] - df['roz_wzr_' + kandydat_B +'_mediana'])/df['roz_wzr_' + kandydat_B + '_MAD']

In [43]:
k = 3

In [44]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 2106)

In [45]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_2']>k)

('NAWROCKI Karol Tadeusz', 1669)

In [46]:
k = 2

In [47]:
kandydat_A, sum(df[kandydat_A +'_k_analiza_2']>k)

('TRZASKOWSKI Rafał Kazimierz', 3552)

In [48]:
kandydat_B, sum(df[kandydat_B +'_k_analiza_2']>k)

('NAWROCKI Karol Tadeusz', 3127)

## WYNIKI:

"Nadmierny" względny wzrost poparcia dla Trzaskowskiego odnotowano w 3552 komisjach

"Nadmierny" względny wzrost poparcia dla Nawrockiego odnotowano w 3127 komisjach

W tej samej grupie co dla analizy 1 mamy jeden obwód z "Nadmiernym względnym wzrostem poparcia" na korzyść Trzaskowskiego (komisja nr 2):

In [49]:
df[df['group_id']==1568]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2
45,20103.0,1,gm. Gromadka,306.0,312.0,gm. Gromadka,423.0,570.0,gm. Gromadka,20103.0,1,59-706,wieś,1628.0,59706,1568,6,91.5,177.0,11.5,32.0,28.826087,12.28125,True,True,1.382353,1.826923,-0.444570,0.444570,-0.431534,0.222619,0.431534,0.222619,-0.058557,0.058557
46,20103.0,2,gm. Gromadka,47.0,208.0,gm. Gromadka,108.0,354.0,gm. Gromadka,20103.0,2,59-706,wieś,770.0,59706,1568,6,91.5,177.0,11.5,32.0,1.434783,5.53125,False,True,2.297872,1.701923,0.595949,-0.595949,-0.431534,0.222619,0.431534,0.222619,4.615441,-4.615441
47,20103.0,3,gm. Gromadka,45.0,59.0,gm. Gromadka,73.0,132.0,gm. Gromadka,20103.0,3,59-706,wieś,305.0,59706,1568,6,91.5,177.0,11.5,32.0,-1.608696,-1.40625,False,False,1.622222,2.237288,-0.615066,0.615066,-0.431534,0.222619,0.431534,0.222619,-0.824422,0.824422
48,20103.0,4,gm. Gromadka,60.0,91.0,gm. Gromadka,85.0,167.0,gm. Gromadka,20103.0,4,59-706,wieś,431.0,59706,1568,6,91.5,177.0,11.5,32.0,-0.565217,-0.31250,False,False,1.416667,1.835165,-0.418498,0.418498,-0.431534,0.222619,0.431534,0.222619,0.058557,-0.058557
49,20103.0,5,gm. Gromadka,49.0,64.0,gm. Gromadka,87.0,158.0,gm. Gromadka,20103.0,5,59-706,wieś,420.0,59706,1568,6,91.5,177.0,11.5,32.0,-0.391304,-0.59375,False,False,1.775510,2.468750,-0.693240,0.693240,-0.431534,0.222619,0.431534,0.222619,-1.175578,1.175578
50,20103.0,6,gm. Gromadka,52.0,111.0,gm. Gromadka,96.0,187.0,gm. Gromadka,20103.0,6,59-706,wieś,436.0,59706,1568,6,91.5,177.0,11.5,32.0,0.391304,0.31250,False,False,1.846154,1.684685,0.161469,-0.161469,-0.431534,0.222619,0.431534,0.222619,2.663762,-2.663762


In [50]:
# anomalia na korzyść kandydata
df[kandydat_A + "_anomalia_2"] = df[kandydat_A +'_k_analiza_2']>2
df[kandydat_B + "_anomalia_2"] = df[kandydat_B +'_k_analiza_2']>2

## 3. Komisje, w których Nawrocki uzyskał więcej głosów niż Trzaskowski w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę Trzaskowskiego;

1. Sprawdzamy, w których grupach dany kandydat miał większą medianę
2. Sumujemy komisje, w których wygrał kandydat A mimo, że większą medianę miał kandydat B i na odwrót.

In [51]:
# wieksza mediana w grupie
df['wieksza_mediana_' + kandydat_A] = (df[kandydat_A + '_mediana_2'] >  df[kandydat_B + '_mediana_2']).astype(bool)
df['wieksza_mediana_' + kandydat_B] = (df[kandydat_B + '_mediana_2'] >  df[kandydat_A + '_mediana_2']).astype(bool)

In [52]:
# na korzyść kandydat A, czyli większą medianę miał B, a więcej głosów dostał A.
kandydat_A, sum(df['wieksza_mediana_' + kandydat_B] & (df[kandydat_A + '_2'] > df[kandydat_B + '_2']))

('TRZASKOWSKI Rafał Kazimierz', 2608)

In [53]:
kandydat_B, sum(df['wieksza_mediana_' + kandydat_A] & (df[kandydat_B + '_2'] > df[kandydat_A + '_2']))

('NAWROCKI Karol Tadeusz', 1843)

## WYNIKI:
W grupach, w których większą medianę miał Nawrocki, było 2608 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1843 komisji, w których wyższy wyniki uzyskał Nawrocki.

Przykładowo:

W komisji 13 gdzie w drugiej turze głosowało.. 13 osób, Trzaskowski uzyskał większy wynik (8 do 5), mimo że w grupie obejmującej kod pocztowy 59-730 większą medianę miał Nawrocki (344 vs 158).

In [54]:
df[df['group_id']==1572]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz
34,20102.0,11,gm. Bolesławiec,81.0,220.0,gm. Bolesławiec,131.0,344.0,gm. Bolesławiec,20102.0,11,59-730,wieś,772.0,59730,1572,15,158.0,344.0,53.0,176.0,-0.509434,0.000000,False,False,1.617284,1.563636,0.053648,-0.053648,-0.174293,0.071987,0.174293,0.071987,3.166427,-3.166427,True,False,False,True
51,20104.0,1,gm. Nowogrodziec,85.0,139.0,gm. Nowogrodziec,158.0,283.0,gm. Nowogrodziec,20104.0,1,59-730,wieś,797.0,59730,1572,15,158.0,344.0,53.0,176.0,0.000000,-0.346591,False,False,1.858824,2.035971,-0.177148,0.177148,-0.174293,0.071987,0.174293,0.071987,-0.039659,0.039659,False,False,False,True
52,20104.0,2,gm. Nowogrodziec,102.0,207.0,gm. Nowogrodziec,161.0,349.0,gm. Nowogrodziec,20104.0,2,59-730,wieś,878.0,59730,1572,15,158.0,344.0,53.0,176.0,0.056604,0.028409,False,False,1.578431,1.685990,-0.107559,0.107559,-0.174293,0.071987,0.174293,0.071987,0.927031,-0.927031,False,False,False,True
53,20104.0,3,gm. Nowogrodziec,72.0,61.0,gm. Nowogrodziec,118.0,133.0,gm. Nowogrodziec,20104.0,3,59-730,wieś,400.0,59730,1572,15,158.0,344.0,53.0,176.0,-0.754717,-1.198864,False,False,1.638889,2.180328,-0.541439,0.541439,-0.174293,0.071987,0.174293,0.071987,-5.100200,5.100200,False,True,False,True
54,20104.0,4,gm. Nowogrodziec,72.0,255.0,gm. Nowogrodziec,175.0,495.0,gm. Nowogrodziec,20104.0,4,59-730,wieś,1107.0,59730,1572,15,158.0,344.0,53.0,176.0,0.320755,0.857955,False,False,2.430556,1.941176,0.489379,-0.489379,-0.174293,0.071987,0.174293,0.071987,9.219379,-9.219379,True,False,False,True
55,20104.0,5,gm. Nowogrodziec,134.0,217.0,gm. Nowogrodziec,211.0,395.0,gm. Nowogrodziec,20104.0,5,59-730,wieś,1012.0,59730,1572,15,158.0,344.0,53.0,176.0,1.000000,0.289773,False,False,1.574627,1.820276,-0.245650,0.245650,-0.174293,0.071987,0.174293,0.071987,-0.991251,0.991251,False,False,False,True
56,20104.0,6,gm. Nowogrodziec,60.0,81.0,gm. Nowogrodziec,109.0,161.0,gm. Nowogrodziec,20104.0,6,59-730,wieś,435.0,59730,1572,15,158.0,344.0,53.0,176.0,-0.924528,-1.039773,False,False,1.816667,1.987654,-0.170988,0.170988,-0.174293,0.071987,0.174293,0.071987,0.045913,-0.045913,False,False,False,True
57,20104.0,7,gm. Nowogrodziec,246.0,369.0,gm. Nowogrodziec,415.0,662.0,gm. Nowogrodziec,20104.0,7,59-730,miasto,1731.0,59730,1572,15,158.0,344.0,53.0,176.0,4.849057,1.806818,True,False,1.686992,1.794038,-0.107046,0.107046,-0.174293,0.071987,0.174293,0.071987,0.934156,-0.934156,False,False,False,True
58,20104.0,8,gm. Nowogrodziec,239.0,297.0,gm. Nowogrodziec,394.0,520.0,gm. Nowogrodziec,20104.0,8,59-730,miasto,1497.0,59730,1572,15,158.0,344.0,53.0,176.0,4.452830,1.000000,True,False,1.648536,1.750842,-0.102306,0.102306,-0.174293,0.071987,0.174293,0.071987,1.000000,-1.000000,False,False,False,True
59,20104.0,9,gm. Nowogrodziec,80.0,86.0,gm. Nowogrodziec,127.0,168.0,gm. Nowogrodziec,20104.0,9,59-730,wieś,497.0,59730,1572,15,158.0,344.0,53.0,176.0,-0.584906,-1.000000,False,False,1.587500,1.953488,-0.365988,0.365988,-0.174293,0.071987,0.174293,0.071987,-2.6

In [55]:
# anomalie na korzysc
df[kandydat_A + '_anomalia_3'] = df['wieksza_mediana_' + kandydat_B] & (df[kandydat_A + '_2'] > df[kandydat_B + '_2']) 
df[kandydat_B + '_anomalia_3'] = df['wieksza_mediana_' + kandydat_A] & (df[kandydat_B + '_2'] > df[kandydat_A + '_2']) 


## 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [56]:
kandydat_A, sum(df[kandydat_A + '_2']<df[kandydat_A + '_1'])

('TRZASKOWSKI Rafał Kazimierz', 128)

In [57]:
kandydat_B, sum(df[kandydat_B + '_2']<df[kandydat_B + '_1'])

('NAWROCKI Karol Tadeusz', 112)

W 128 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

W 112 komisjach Nawrocki uzyskał mniej głosó w drugiej turze niż w pierwszej.

Przykładowe anomalie na korzyść Trzaskowskiego:

In [58]:
df[df[kandydat_B + '_2']<df[kandydat_B + '_1']].head()

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz,TRZASKOWSKI Rafał Kazimierz_anomalia_3,NAWROCKI Karol Tadeusz_anomalia_3
125,20202.0,23,m. Dzierżoniów,8.0,9.0,m. Dzierżoniów,12.0,7.0,m. Dzierżoniów,20202.0,23,58-200,miasto,47.0,58200,1516,24,383.5,302.0,74.0,49.0,-5.020270,-6.020408,False,False,1.500000,0.777778,0.722222,-0.722222,-0.235163,0.133249,0.235163,0.133249,7.184909,-7.184909,True,False,True,False,False,False
321,20602.0,8,m. Kowary,22.0,25.0,m. Kowary,26.0,19.0,m. Kowary,20602.0,8,58-530,miasto,48.0,58530,1543,10,329.0,216.5,161.0,91.0,-1.881988,-2.170330,False,False,1.181818,0.760000,0.421818,-0.421818,-0.304523,0.200856,0.304523,0.200856,3.616234,-3.616234,True,False,True,False,False,False
360,20608.0,7,gm. Podgórzyn,4.0,15.0,gm. Podgórzyn,7.0,13.0,gm. Podgórzyn,20608.0,7,58-564,wieś,55.0,58564,1546,16,483.5,300.5,309.5,111.0,-1.539580,-2.590090,False,False,1.750000,0.866667,0.883333,-0.883333,-0.250784,0.187909,0.250784,0.187909,6.035457,-6.035457,True,False,True,False,False,True
375,20701.0,10,m. Kamienna Góra,6.0,7.0,m. Kamienna Góra,10.0,2.0,m. Kamienna Góra,20701.0,10,58-400,miasto,37.0,58400,1537,23,145.0,154.0,86.0,89.0,-1.569767,-1.707865,False,False,1.666667,0.285714,1.380952,-1.380952,-0.276397,0.203887,0.276397,0.203887,8.128783,-8.128783,True,False,False,True,True,False
636,21002.0,5,m. Świeradów-Zdrój,7.0,9.0,m. Świeradów-Zdrój,5.0,8.0,m. Świeradów-Zdrój,21002.0,5,59-850,miasto,21.0,59850,2714,11,98.0,211.0,73.0,53.0,-1.273973,-3.830189,False,False,0.714286,0.888889,-0.174603,0.174603,-0.271933,0.407785,0.271933,0.407785,0.238679,-0.238679,False,False,False,True,False,False


In [59]:
# Anomalie na korzysc
df[kandydat_A + '_anomalia_4'] = df[kandydat_B + '_2']<df[kandydat_B + '_1']
df[kandydat_B + '_anomalia_4'] = df[kandydat_A + '_2']<df[kandydat_A + '_1']

In [60]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df.dropna()

# Sumowanie anomalii

# Na korzyść Trzaskowskiego

In [61]:
df[kandydat_A + '_razem_anomali_na_korzysc'] = df[[
    kandydat_A + '_anomalia_1', 
    kandydat_A + '_anomalia_2',
    kandydat_A + '_anomalia_3',
    kandydat_A + '_anomalia_4']].sum(axis=1)

In [62]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=1)

('TRZASKOWSKI Rafał Kazimierz', 9386)

In [63]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=2)

('TRZASKOWSKI Rafał Kazimierz', 1276)

In [64]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=3)

('TRZASKOWSKI Rafał Kazimierz', 43)

In [65]:
kandydat_A, sum(df[kandydat_A + '_razem_anomali_na_korzysc']>=4)

('TRZASKOWSKI Rafał Kazimierz', 3)

In [66]:
# Komisje z wszystkimi czterma anomaliami
df[df[kandydat_A + '_razem_anomali_na_korzysc']>=4]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz,TRZASKOWSKI Rafał Kazimierz_anomalia_3,NAWROCKI Karol Tadeusz_anomalia_3,TRZASKOWSKI Rafał Kazimierz_anomalia_4,NAWROCKI Karol Tadeusz_anomalia_4,TRZASKOWSKI Rafał Kazimierz_razem_anomali_na_korzysc
4994,60903.0,4,gm. Bychawa,89.0,174.0,gm. Bychawa,260.0,163.0,gm. Bychawa,60903.0,4,23-100,miasto,659.0,23100,522,20,64.0,203.5,33.5,68.0,5.850746,-0.595588,True,False,2.921348,0.936782,1.984567,-1.984567,0.283991,0.291965,-0.283991,0.291965,5.824589,-5.824589,True,False,False,True,True,False,True,False,4
12294,140706.0,1,gm. Magnuszew,105.0,285.0,gm. Magnuszew,467.0,193.0,gm. Magnuszew,140706.0,1,26-910,wieś,878.0,26910,630,10,102.0,221.0,42.5,36.5,8.588235,-0.767123,True,False,4.447619,0.677193,3.770426,-3.770426,0.106758,0.364758,-0.106758,0.364758,10.044104,-10.044104,True,False,False,True,True,False,True,False,4
24999,261207.0,4,gm. Staszów,143.0,224.0,gm. Staszów,360.0,209.0,gm. Staszów,261207.0,4,28-200,miasto,808.0,28200,662,29,123.0,294.0,97.0,57.0,2.443299,-1.491228,True,False,2.517483,0.933036,1.584447,-1.584447,0.020685,0.200502,-0.020685,0.200502,7.799246,-7.799246,True,False,False,True,True,False,True,False,4


# Na korzyść Nawrockiego

In [67]:
df[kandydat_B + '_razem_anomali_na_korzysc'] = df[[
    kandydat_B + '_anomalia_1', 
    kandydat_B + '_anomalia_2',
    kandydat_B + '_anomalia_3',
    kandydat_B + '_anomalia_4']].sum(axis=1)

In [68]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=1)

('NAWROCKI Karol Tadeusz', 8028)

In [69]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=2)

('NAWROCKI Karol Tadeusz', 689)

In [70]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=3)

('NAWROCKI Karol Tadeusz', 42)

In [71]:
kandydat_B, sum(df[kandydat_B + '_razem_anomali_na_korzysc']>=4)

('NAWROCKI Karol Tadeusz', 2)

In [72]:
# Komisje z 4 anomaliami, "widać Kraków"
df[df[kandydat_B + '_razem_anomali_na_korzysc']>=4]

,Teryt Gminy,Nr komisji,Gmina_1,TRZASKOWSKI Rafał Kazimierz_1,NAWROCKI Karol Tadeusz_1,Gmina_2,TRZASKOWSKI Rafał Kazimierz_2,NAWROCKI Karol Tadeusz_2,Gmina,TERYT gminy,Numer,Kod pocztowy,Typ obszaru,Wyborcy,kod,group_id,liczba komisji w grupie,TRZASKOWSKI Rafał Kazimierz_mediana_2,NAWROCKI Karol Tadeusz_mediana_2,TRZASKOWSKI Rafał Kazimierz_MAD_2,NAWROCKI Karol Tadeusz_MAD_2,TRZASKOWSKI Rafał Kazimierz_k_analiza_1,NAWROCKI Karol Tadeusz_k_analiza_1,TRZASKOWSKI Rafał Kazimierz_anomalia_1,NAWROCKI Karol Tadeusz_anomalia_1,TRZASKOWSKI Rafał Kazimierz_wzrost,NAWROCKI Karol Tadeusz_wzrost,roz_wzr_TRZASKOWSKI Rafał Kazimierz,roz_wzr_NAWROCKI Karol Tadeusz,roz_wzr_TRZASKOWSKI Rafał Kazimierz_mediana,roz_wzr_TRZASKOWSKI Rafał Kazimierz_MAD,roz_wzr_NAWROCKI Karol Tadeusz_mediana,roz_wzr_NAWROCKI Karol Tadeusz_MAD,TRZASKOWSKI Rafał Kazimierz_k_analiza_2,NAWROCKI Karol Tadeusz_k_analiza_2,TRZASKOWSKI Rafał Kazimierz_anomalia_2,NAWROCKI Karol Tadeusz_anomalia_2,wieksza_mediana_TRZASKOWSKI Rafał Kazimierz,wieksza_mediana_NAWROCKI Karol Tadeusz,TRZASKOWSKI Rafał Kazimierz_anomalia_3,NAWROCKI Karol Tadeusz_anomalia_3,TRZASKOWSKI Rafał Kazimierz_anomalia_4,NAWROCKI Karol Tadeusz_anomalia_4,TRZASKOWSKI Rafał Kazimierz_razem_anomali_na_korzysc,NAWROCKI Karol Tadeusz_razem_anomali_na_korzysc
11295,126101.0,95,m. Kraków,550.0,218.0,m. Kraków,540.0,1132.0,m. Kraków,126101.0,95,31-346,miasto,1943.0,31346,691,14,814.5,424.5,174.0,46.0,-1.577586,15.380435,False,True,0.981818,5.192661,-4.210842,4.210842,-0.059256,0.158571,0.059256,0.158571,-26.181205,26.181205,False,True,True,False,False,True,False,True,0,4
16574,161105.0,9,gm. Strzelce Opolskie,311.0,107.0,gm. Strzelce Opolskie,223.0,416.0,gm. Strzelce Opolskie,161105.0,9,47-100,miasto,957.0,47100,1408,34,230.5,174.5,101.0,60.0,-0.074257,4.025000,False,True,0.717042,3.887850,-3.170809,3.170809,-0.656285,0.240919,0.656285,0.240919,-10.437204,10.437204,False,True,True,False,False,True,False,True,0,4
